In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

# 读取股票数据
file_path = "jq_stock_ticks_next.csv"
stock_data = pd.read_csv(file_path)

# 定义预测目标的偏移量（1000个tick后）
offset = 1000

# 为训练目标（Y）创建一个新列，该列将包含未来1000个tick的均值
stock_data['target'] = stock_data['current'].rolling(window=offset).mean().shift(-offset)

# 删除最后1000行，因为这些行没有目标值
stock_data = stock_data[:-offset]

# 选择特征（除了时间和目标外的所有列）
X = stock_data.drop(columns=['time', 'target'])

# 选择目标
y = stock_data['target']

# 按80%和20%的比例拆分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=False)

# 创建随机森林回归器实例
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# 训练模型
rf_regressor.fit(X_train, y_train)

# 预测测试集的股价
y_pred = rf_regressor.predict(X_test)

# 初始化投资组合的变量
portfolio_value = 100000  # 初始投资
cash = portfolio_value
stock_quantity = 0

# 设定买卖阈值
threshold = 0.05

# 模拟交易
for i in range(len(y_test) - offset):
    # 计算预测的涨跌幅
    predicted_change = (y_pred[i + offset] - X_test['current'].iloc[i]) / X_test['current'].iloc[i]
    
    # 获取当前股价
    current_price = X_test['current'].iloc[i]
    
    # 获取当前时间
    current_time = stock_data['time'].iloc[X_train.shape[0] + i]
    
    # 如果预测涨幅超过阈值，则买入
    if predicted_change > threshold:
        # 全部买入
        stock_quantity += cash / current_price
        cash = 0
        print("Buy at time:", current_time, "Price:", current_price)
        
    # 如果预测跌幅超过阈值，则卖出
    elif predicted_change < -threshold:
        # 全部卖出
        cash += stock_quantity * current_price
        stock_quantity = 0
        print("Sell at time:", current_time, "Price:", current_price)

# 计算最终的投资组合价值
final_portfolio_value = cash + stock_quantity * X_test['current'].iloc[-offset]
profit_rate = (final_portfolio_value - portfolio_value) / portfolio_value

print("Profit Rate:", profit_rate * 100, "%")  # 将收益率转换为百分比


Buy at time: 2023-04-26 09:30:09 Price: 1.32
Buy at time: 2023-04-26 09:30:12 Price: 1.32
Buy at time: 2023-04-26 09:30:15 Price: 1.32
Buy at time: 2023-04-26 09:30:18 Price: 1.32
Buy at time: 2023-04-26 09:30:21 Price: 1.32
Buy at time: 2023-04-26 09:30:24 Price: 1.32
Buy at time: 2023-04-26 09:30:27 Price: 1.32
Buy at time: 2023-04-26 09:30:30 Price: 1.32
Buy at time: 2023-04-26 09:30:33 Price: 1.32
Buy at time: 2023-04-26 09:30:36 Price: 1.32
Buy at time: 2023-04-26 09:30:39 Price: 1.32
Buy at time: 2023-04-26 09:30:45 Price: 1.32
Buy at time: 2023-04-26 09:30:48 Price: 1.32
Buy at time: 2023-04-26 09:30:51 Price: 1.32
Buy at time: 2023-04-26 09:30:54 Price: 1.32
Buy at time: 2023-04-26 09:30:57 Price: 1.32
Buy at time: 2023-04-26 09:31:00 Price: 1.32
Buy at time: 2023-04-26 09:31:03 Price: 1.32
Buy at time: 2023-04-26 09:31:06 Price: 1.32
Buy at time: 2023-04-26 09:31:09 Price: 1.31
Buy at time: 2023-04-26 09:31:12 Price: 1.32
Buy at time: 2023-04-26 09:31:15 Price: 1.32
Buy at tim